<a href="https://colab.research.google.com/github/HofstraDoboli/TextMining/blob/main/page_rank.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np

n = 4
mat_con = np.array([(0, 0, 1, 1), (1,0,0,0), (0,1,0,0), (1,1,0,0)])
sum_con = mat_con.sum(axis = 1)   # sum over the rows
print(mat_con)
print(sum_con)
#sum_con[sum_con == 0] = 1
#print(sum_con)
prob_mat_con = np.divide(mat_con,sum_con[:,np.newaxis])  # add a new axis to sum_con
print(prob_mat_con)
random_mat = (1/n) * np.ones((n,n))   # creates a matrix of ones
print(random_mat)

[[0 0 1 1]
 [1 0 0 0]
 [0 1 0 0]
 [1 1 0 0]]
[2 1 1 2]
[[0.  0.  0.5 0.5]
 [1.  0.  0.  0. ]
 [0.  1.  0.  0. ]
 [0.5 0.5 0.  0. ]]
[[0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]
 [0.25 0.25 0.25 0.25]]


In [ ]:
# initialize rank
p0 = 0.1 * np.ones((4,1)) # np.random.rand(4,1)
print(p0)
# initialize an array with random values - uniform distributed 0 to 1

p_current = np.copy(p0)
alpha = 0.3

A_mat = np.transpose(alpha * random_mat + (1- alpha) * prob_mat_con)
print(A_mat)
p_new  = A_mat @ p_current  # (4,4) X(4,1)= (4,1)
iter = 1
while np.any(np.absolute(p_current - p_new) > 0.001):
    p_current = p_new
    p_new = A_mat @ p_current
    print("\nIteration", iter)
    print("Previous value\n", p_current)
    print("New value\n", p_new)
    iter += 1

print(p_new)

[[0.1]
 [0.1]
 [0.1]
 [0.1]]
[[0.075 0.775 0.075 0.425]
 [0.075 0.075 0.775 0.425]
 [0.425 0.075 0.075 0.075]
 [0.425 0.075 0.075 0.075]]

Iteration 1
Previous value
 [[0.135]
 [0.135]
 [0.065]
 [0.065]]
New value
 [[0.14725]
 [0.09825]
 [0.07725]
 [0.07725]]

Iteration 2
Previous value
 [[0.14725]
 [0.09825]
 [0.07725]
 [0.07725]]
New value
 [[0.1258125]
 [0.1111125]
 [0.0815375]
 [0.0815375]]

Iteration 3
Previous value
 [[0.1258125]
 [0.1111125]
 [0.0815375]
 [0.0815375]]
New value
 [[0.13631687]
 [0.11561437]
 [0.07403437]
 [0.07403437]]

Iteration 4
Previous value
 [[0.13631687]
 [0.11561437]
 [0.07403437]
 [0.07403437]]
New value
 [[0.13684209]
 [0.10773609]
 [0.07771091]
 [0.07771091]]

Iteration 5
Previous value
 [[0.13684209]
 [0.10773609]
 [0.07771091]
 [0.07771091]]
New value
 [[0.13261408]
 [0.11159645]
 [0.07789473]
 [0.07789473]]

Iteration 6
Previous value
 [[0.13261408]
 [0.11159645]
 [0.07789473]
 [0.07789473]]
New value
 [[0.13538067]
 [0.11178947]
 [0.07641493]
 [0.0

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [2]:
%cd /content/drive/MyDrive/TextMining/

/content/drive/MyDrive/TextMining


In [3]:
!pip install wikipedia-api

  Preparing metadata (setup.py) ... done
  Created wheel for wikipedia-api: filename=Wikipedia_API-0.8.1-py3-none-any.whl size=15383 sha256=af060cf6c847a353c09e94d2815a11dd2f6077a13fa2484903a0711585eca86d
  Stored in directory: /root/.cache/pip/wheels/33/3c/79/b36253689d838af4a0539782853ac3cc38a83a6591ad570dde
Successfully built wikipedia-api


In [4]:
import wikipediaapi
import networkx as nx

wiki = wikipediaapi.Wikipedia(language='en', user_agent='MyWikipediaApp/1.0') # Added user agent
category = wiki.page("Category:Information retrieval") # starting web page
G = nx.DiGraph()

for title in category.categorymembers.keys(): # for all links in the starting web page
    page = wiki.page(title)
    for link in page.links.keys(): # extract titles for each link in the starting web page
        G.add_edge(title, link)
        #subpage = wiki.page(link)
        #for sublink in subpage.links.keys():
        #  G.add_edge(link, sublink)

print(G)
print("Number of nodes:", G.number_of_nodes())
print("Number of edges:", G.number_of_edges())

DiGraph with 644 nodes and 714 edges
Number of nodes: 644
Number of edges: 714


In [5]:
# Save the graph to a GraphML file
nx.write_graphml(G, "information_retrieval_graph.graphml")
print("Graph saved")

Graph saved


In [8]:
import networkx as nx

# Load the graph from the GraphML file
G = nx.read_graphml("information_retrieval_graph.graphml")
print("Graph loaded successfully.")
print(G)
print("Number of nodes in loaded graph:", G.number_of_nodes())
print("Number of edges in loaded graph:", G.number_of_edges())

Graph loaded successfully.
DiGraph with 644 nodes and 714 edges
Number of nodes in loaded graph: 644
Number of edges in loaded graph: 714


In [17]:
import numpy as np

def pagerank(graph, alpha=0.85, tol=1e-6, max_iter=100):
    """
    Implements the PageRank algorithm on a NetworkX graph.

    Args:
        graph: A NetworkX graph.
        alpha: The damping factor (probability of following a link).
        tol: The tolerance for convergence.
        max_iter: The maximum number of iterations.

    Returns:
        A dictionary of PageRank scores for each node.
    """
    nodes = list(graph.nodes())
    n = len(nodes)
    # Create a mapping from node to index
    node_to_index = {node: i for i, node in enumerate(nodes)}

    # Build the transition matrix T from the adjacency matrix
    # Handle dangling nodes (nodes with no outgoing edges)
    out_degree = np.array([graph.out_degree(node) for node in nodes])
    # Replace 0 out_degree with 1 to avoid division by zero, these nodes will distribute their rank equally
    out_degree[out_degree == 0] = 1

    # Create the transition matrix
    # The original graph adjacency matrix is G, where G[i,j] = 1 if there is an edge from i to j
    # The transition matrix T has T[j,i] = 1/out_degree[i] if there is an edge from i to j
    # We need to build T from the adjacency matrix of the graph
    adj_matrix = nx.adjacency_matrix(graph, nodelist=nodes).todense().T # Transpose to get correct transitions

    # Normalize the adjacency matrix by out_degree
    # Use broadcasting to divide each column by the out_degree of the source node (column sum before transpose)
    # We need to divide by out-degree of the source node, which is the sum of rows in the non-transposed matrix
    # Let's re-calculate transition matrix directly based on definition
    T = np.zeros((n, n))
    for u in graph.nodes():
        for v in graph.successors(u):
            T[node_to_index[v], node_to_index[u]] = 1.0 / graph.out_degree(u)

    # Create the Google matrix G
    # Handling dangling nodes: distribute their rank equally among all nodes
    dangling_nodes_index = [i for i, node in enumerate(nodes) if graph.out_degree(node) == 0]
    A_prime = alpha * T
    # Add contribution from dangling nodes and random jumps
    A_prime[:, dangling_nodes_index] += alpha * (1/n)
    G_matrix = A_prime + (1 - alpha) * (1/n)


    # Initialize PageRank vector
    p_current = np.ones((n, 1)) / n

    # Iterative power method
    for i in range(max_iter):
        p_new = G_matrix @ p_current
        if np.linalg.norm(p_new - p_current, ord=1) < tol:
            print(f"PageRank converged after {i+1} iterations.")
            break
        p_current = p_new
    else:
        print(f"PageRank did not converge after {max_iter} iterations.")

    # Convert PageRank vector to a dictionary with node names
    pagerank_scores = {nodes[i]: p_current[i, 0] for i in range(n)}

    return pagerank_scores

ranked_scores = dict(sorted(pagerank_scores.items(), key=lambda item: item[1], reverse=True)[:20])
# Print the PageRank scores
print("PageRank scores:")
for node, score in ranked_scores.items():
    print(f"Node: {node}, PageRank Score: {score:.4f}")

PageRank scores:
Node: Information retrieval, PageRank Score: 0.0035
Node: Help:Categories, PageRank Score: 0.0028
Node: Web search engine, PageRank Score: 0.0022
Node: ISBN (identifier), PageRank Score: 0.0020
Node: Information seeking, PageRank Score: 0.0020
Node: Citation index, PageRank Score: 0.0019
Node: Data structure, PageRank Score: 0.0018
Node: Inverted index, PageRank Score: 0.0018
Node: Category:Bibliographic databases and indexes, PageRank Score: 0.0018
Node: Category:Citation metrics, PageRank Score: 0.0018
Node: Information behavior, PageRank Score: 0.0018
Node: Search engine, PageRank Score: 0.0018
Node: Library classification, PageRank Score: 0.0018
Node: Semantic network, PageRank Score: 0.0018
Node: Help:Disambiguation, PageRank Score: 0.0018
Node: Knowledge representation, PageRank Score: 0.0018
Node: Ontology (computer science), PageRank Score: 0.0018
Node: Doi (identifier), PageRank Score: 0.0017
Node: Library and information science, PageRank Score: 0.0017
Node: 

In [13]:
# Apply the PageRank algorithm
pagerank_scores = nx.pagerank(loaded_graph)

ranked_scores = dict(sorted(pagerank_scores.items(), key=lambda item: item[1], reverse=True)[:20])
# Print the PageRank scores
print("PageRank scores:")
for node, score in ranked_scores.items():
    print(f"Node: {node}, PageRank Score: {score:.4f}")

PageRank scores:
Node: Information retrieval, PageRank Score: 0.0035
Node: Help:Categories, PageRank Score: 0.0028
Node: Web search engine, PageRank Score: 0.0022
Node: ISBN (identifier), PageRank Score: 0.0020
Node: Information seeking, PageRank Score: 0.0020
Node: Citation index, PageRank Score: 0.0019
Node: Data structure, PageRank Score: 0.0018
Node: Inverted index, PageRank Score: 0.0018
Node: Category:Bibliographic databases and indexes, PageRank Score: 0.0018
Node: Category:Citation metrics, PageRank Score: 0.0018
Node: Information behavior, PageRank Score: 0.0018
Node: Search engine, PageRank Score: 0.0018
Node: Library classification, PageRank Score: 0.0018
Node: Semantic network, PageRank Score: 0.0018
Node: Help:Disambiguation, PageRank Score: 0.0018
Node: Knowledge representation, PageRank Score: 0.0018
Node: Ontology (computer science), PageRank Score: 0.0018
Node: Doi (identifier), PageRank Score: 0.0017
Node: Library and information science, PageRank Score: 0.0017
Node: 

In [29]:
# Iterate through the nodes and print their names
print("Nodes in the graph:")
count = 0
for node in G.nodes():
    if count < 10:
      print(count, node)
      count += 1
    else:
      break

Nodes in the graph:
0 Information retrieval
1 1890 US census
2 1790 United States census
3 1800 United States census
4 1810 United States census
5 1820 United States census
6 1830 United States census
7 1840 United States census
8 1850 United States census
9 1860 United States census


In [14]:
# print edge information for each node
print("Edge information for the first 10 nodes:")
count = 0
for node in G.nodes():
    if count < 10:
        print(f"\nNode: {node}")
        print(f"  Successors (outgoing edges): {list(G.successors(node))}")
        print(f"  Predecessors (incoming edges): {list(G.predecessors(node))}")
        count += 1
    else:
        break

Edge information for the first 10 nodes:

Node: Information retrieval
  Successors (outgoing edges): ['1890 US census', '3D retrieval', 'Adversarial information retrieval', 'Allen Kent', 'AltaVista', 'Alvin Weinberg', 'Apache Lucene', 'Apache Solr', 'ArXiv (identifier)', 'As We May Think', 'Association for Computing Machinery', 'Atlantic Monthly', 'Automatic summarization', 'BERT (language model)', "Bayes' theorem", 'Bibliometrics', 'Bill Maron', 'Binary Independence Model', 'C. J. van Rijsbergen', 'CERN', 'Calvin Mooers', 'Case Western Reserve University', 'Categorization', 'Censorship', 'Citation index', 'CiteSeerX (identifier)', 'Co-occurrence', 'Collaborative information seeking', 'Communications of the ACM', 'Compound term processing', 'Computational linguistics', 'Computer data storage', 'Computer memory', 'Computing', 'Conference on Information and Knowledge Management', 'Controlled vocabulary', 'Cornelis J. van Rijsbergen', 'Cross-language information retrieval', 'Cultural stud

In [15]:
# Number of successors for each node
print("Number of successors for each node:")
count = 0
for node in G.nodes():
    if count < 10:
        print(f"Node: {node}, Number of Successors: {G.out_degree(node)}")
        print(f"Node: {node}, Number of Predecessors: {G.in_degree(node)}")
        count += 1
    else:
        break

Number of successors for each node:
Node: Information retrieval, Number of Successors: 220
Node: Information retrieval, Number of Predecessors: 5
Node: 1890 US census, Number of Successors: 0
Node: 1890 US census, Number of Predecessors: 1
Node: 3D retrieval, Number of Successors: 0
Node: 3D retrieval, Number of Predecessors: 1
Node: Adversarial information retrieval, Number of Successors: 0
Node: Adversarial information retrieval, Number of Predecessors: 1
Node: Allen Kent, Number of Successors: 0
Node: Allen Kent, Number of Predecessors: 1
Node: AltaVista, Number of Successors: 0
Node: AltaVista, Number of Predecessors: 1
Node: Alvin Weinberg, Number of Successors: 0
Node: Alvin Weinberg, Number of Predecessors: 1
Node: Apache Lucene, Number of Successors: 0
Node: Apache Lucene, Number of Predecessors: 1
Node: Apache Solr, Number of Successors: 0
Node: Apache Solr, Number of Predecessors: 1
Node: ArXiv (identifier), Number of Successors: 0
Node: ArXiv (identifier), Number of Predeces